- Scikit pipeline?
- Napravi grader da proceni koliko je tezak primer i primeni na svim
- Import podataka -> Train/Valid split N puta -> Generisanje suma samo za train (da bi mogli da se uverimo da zaista daje dobre rezultate, ako poremetimo valid set mozda greskom dobijemo dobre ili lose rez.) -> 
    Transformacije (SkewTest+Norm/Stand, SMOTE, Method specific transf) - train na train, apply na valid ->
    Trening na train, validacija na valid -> viz i cuvanje
- Sacuvaj - za svaku kombinaciju transf, modela i generisanje suma po jedan entry u pd da bi uzeli najbolje
- Vizuelizacije - Preciznost standardna za svaki difficulty + broj pogodjenih?, za difficulty-e, grafikon za top 5-10 metoda kao linija, grafikon gde su outlieri x-ovi ali zeleni su dobro a crveni lose klasifikovani
- Ako izbacujes ispisi koliko lakih, srednjih, teskih je izbaceno

# Import

In [ ]:
%pip install kagglehub openpyxl imbalanced-learn seaborn torch

In [2]:
import kagglehub as kg
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_predict
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, LabelEncoder

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import VotingClassifier

from helper.plot import plot_attr_label, plot_attributes, plot_difficulties
from helper.transform import transform_pca, remove_outliers_zscore, remove_outliers_db, remove_outliers_isf, bin_attributes_mean, bin_attributes_median, regression_reduce_noise
from helper.transform import remove_label_noise_ensemble_filter, remove_label_noise_cross_validated_committees_filter, remove_label_noise_iterative_partitioning_filter
from helper.train import run_models, test_models, difficulty_split
from helper.noise import add_attribute_noise

from sklearn.base import ClassifierMixin
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

from sklearn.base import TransformerMixin
from typing import Literal, Callable

# Load data

In [ ]:
path = kg.dataset_download("muratkokludataset/dry-bean-dataset")
file_name = '/Dry_Bean_Dataset/Dry_Bean_Dataset.xlsx'
print("Downloaded at: ", path)
data = pd.read_excel(path+file_name)
data

In [ ]:
def encode_n_split(data: pd.DataFrame, train_percentage: float, encoder: TransformerMixin, class_column:str) -> tuple[pd.DataFrame, pd.DataFrame, str] :
    """
        returns tuple containing train set, test set and string representing the label column and the trained encoder
    """
    datacp = data.copy()
    encoded_class_column = f"Class_{encoder.__class__.__name__}"
    datacp[encoded_class_column] = encoder.fit_transform(datacp[class_column])
    datacp.drop(inplace=True, axis=1, labels=[class_column])

    test_percentage = 1 - train_percentage

    split = StratifiedShuffleSplit(n_splits=1, test_size=test_percentage, random_state=42)

    # Perform the split
    for train_idx, test_idx in split.split(datacp, datacp[encoded_class_column]):
        train_set = datacp.iloc[train_idx]
        test_set = datacp.iloc[test_idx]

    print('Train size: ', len(train_set), 'x', len(train_set.iloc[0]))
    print('Test size: ', len(test_set), 'x', len(test_set.iloc[0]))
    print(f'Encoded classes in column "{encoded_class_column}"')
    return (train_set, test_set, encoder, encoded_class_column)


In [ ]:
train_set, test_set, encoder, encoded_class_column = encode_n_split(data, 0.8, LabelEncoder(), 'Class')

In [ ]:
y_train = train_set[encoded_class_column]
y_test = test_set[encoded_class_column]
X_train = train_set.drop(axis = 1, labels=[encoded_class_column])
X_test = test_set.drop(axis = 1, labels=[encoded_class_column])

# EDA

In [ ]:
n_classes, class_counts = np.unique(y_train, return_counts = True)

In [ ]:
from helper.plot import plot_attr_label, plot_attributes, plot_difficulties
import seaborn as sns

In [ ]:
plot_attributes(X_train, label_column = encoded_class_column, n_attrs = len(class_counts))

Atributi 
- Area
- Perimeter
- Major Axis Length
- Minor Axis Length
- AspectRation
- ConvexArea
- EquivDiameter 

imaju dosta velike vrednosti dok ostali atributi su u range $[0-1]$ \
Sa grafikona vidimo da vrednosti atributa ne prate normalnu raspodelu.\
Atributi kao ShapeFactor4 i Solidity imaju velike repove. Zato ćemo tokom preprocesiranja normalizovati raspodele.

In [ ]:
_ = sns.countplot(data = y_train.to_frame(), x = encoded_class_column)

Pored toga skup podataka nije balansiran:
- Klasa 1 je slabo zastupljena sa manje od 500 instanci, klasa 0 ima oko 1000 dok klasa 3 dominira sa oko 2500 instanci
- Zato ćemo izvršiti under i oversampling na oko 500-1000 instanci

In [ ]:
import matplotlib.pyplot as plt
sns.pairplot(pd.concat([X_train, y_train.rename('Class')], axis=1), hue='Class')
plt.show()

Po grafikonu iznad vidimo da se klasa 1 karakteriše visokim vrednostima Major i Minor axis length, EquivDiameter, ConvexArea, Area i Perimeter. Pored toga ima male vrednosti ShapeFactor1.

<div style="background-color:#1212AA; height:auto; border-radius:10px; padding:16px; width:600px; color:white">
<h3>Zaključci</h3>
<ul>
<li>Podaci su nebalansirani, potrebno je under i over sample-ovati na oko 1-1.5 hiljade instanci</li>
<li>Atribute je potrebno normalizovati, a neke i skalirati kao što su Area</li>
<li>Klasa 1 je nedovoljno zastupljena ali lako prepoznatljiva po atributima koji imaju visoke vrednosti kao što je Area</li>
</ul>
</div>


# Normalizacija/Standardizacija

In [ ]:
pt = PowerTransformer(method='yeo-johnson')
X_train_transformed = pt.fit_transform(X_train)

In [ ]:
# Samo power transform
X_train_transformed = pd.DataFrame(X_train_transformed, columns=X_train.columns)
sns.pairplot(pd.concat([X_train_transformed, y_train.rename('Class')], axis=1), hue='Class')
plt.show()

In [ ]:
# PowerTransform + MinMaxScaler
scaler = MinMaxScaler()
train_set_scaled = scaler.fit_transform(X_train_transformed)
X_train_transformed = pd.DataFrame(train_set_scaled, columns=X_train.columns)
sns.pairplot(pd.concat([X_train_transformed, y_train.rename('Class')], axis=1), hue='Class')
plt.show()

In [ ]:
# Samo MinMaxScaler
scaler = MinMaxScaler()
train_set_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(train_set_scaled, columns=X_train.columns)
sns.pairplot(pd.concat([X_train_scaled, y_train.rename('Class')], axis=1), hue='Class')
plt.show()

In [ ]:
def TestClassifiers(
    dataPredictorPairs: list[tuple[pd.DataFrame, pd.DataFrame, ClassifierMixin, str]],
    n_columns=4,
    cv=5,
    normalize = "pred"
):
    test_results = []
    n_rows = len(dataPredictorPairs) // n_columns
    if n_rows == 0:
        n_rows = 1
        n_columns = len(dataPredictorPairs)
    elif n_rows * n_columns < len(dataPredictorPairs):
        n_rows += 1

    _, axes = plt.subplots(n_rows, n_columns, figsize=(5 * n_columns, 4 * n_rows))
    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]

    for i, (X, y, predictor, test_name) in enumerate(dataPredictorPairs):
        # Get cross-validated predictions
        y_pred = cross_val_predict(predictor, X, y, cv=cv)
        accuracy = round(accuracy_score(y, y_pred), 2)
        cm = confusion_matrix(y, y_pred, normalize=normalize)
        test_results.append((test_name, accuracy, cm))

        ConfusionMatrixDisplay(cm).plot(ax=axes[i], colorbar=False)
        axes[i].set_title(f"{test_name}\nCV Accuracy: {accuracy:.2f}")

    # Hide unused axes
    for j in range(len(dataPredictorPairs), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()

Na osnovu prethodnih grafikona mozemo da zakljucimo da `PowerTransform` i `MinMaxScaler` pogorsavaju separaciju razlicitih klasa.\

In [ ]:
TestClassifiers(
    dataPredictorPairs=[
        (X_train, y_train, RandomForestClassifier(n_jobs=-1, n_estimators=20), "Without transforms"),
        (X_train_transformed, y_train, RandomForestClassifier(n_jobs=-1, n_estimators=20), "With power and scaling transforms"),
        (X_train_scaled, y_train, RandomForestClassifier(n_jobs=-1, n_estimators=20), "With scaling only"),
    ], 
)

In [ ]:
TestClassifiers(
    dataPredictorPairs=[
        (X_train, y_train, SVC(), "Without transforms"),
        (X_train_transformed, y_train, SVC(), "With power and scaling transforms"),
        (X_train_scaled, y_train, SVC(), "With scaling only"),
    ],
)

Međutim, rezultati pokazuju bolji accuracy kada se primeni Power i Scaling transformacija. Ovo se vidi i po accuracy score-u kao i po konfuzionoj matrici.\
Zato na dalje koristimo transformisani skup podataka.\
Iako ima malo instanci klase 1, klasifikacija ovih instanci je jednostavna za oba modela.
Sa druge strane, iako model ima najviše instanci klase 3 ova klasa je najteža za klasifikovati. Ovo ukazuje na overfitting ili šum među instancama klase 3.

# Under/OverSampling

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

target_samples = 1700 # Pick a number in the range

# Oversample with SMOTE and undersample with RandomUnderSampler
smote = SMOTE(sampling_strategy=lambda y: {k: max(target_samples, v) for k, v in Counter(y).items() if v < target_samples}, random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_transformed, y_train)

under = RandomUnderSampler(sampling_strategy=lambda y: {k: min(target_samples, v) for k, v in Counter(y).items() if v > target_samples}, random_state=42)
X_train_resampled, y_train_resampled = under.fit_resample(X_train_resampled, y_train_resampled)

In [ ]:
_ = sns.countplot(data = y_train_resampled.to_frame(), x = encoded_class_column)

In [ ]:
TestClassifiers(
    dataPredictorPairs=[
        (X_train_transformed, y_train, RandomForestClassifier(n_jobs=-1, n_estimators=20), "RandomForest, No Over/Undersample"),
        (X_train_resampled, y_train_resampled, RandomForestClassifier(n_jobs=-1, n_estimators=20), "RandomForest + SMOTE + Undersample"),
        (X_train_transformed, y_train, SVC(), "SVC, No Over/Undersample"),
        (X_train_resampled, y_train_resampled, SVC(), "SVC + SMOTE + Undersample"),
    ], 
)

Vršenjem under i oversamplinga dobijamo malo bolje rezultate, pa na dalje koristimo `X_train_resampled`

# Difficulty split

In [ ]:
def difficulty_split(X, y, models, difficulty_categories=3, cv=5):
    """
    Estimate difficulty of each sample based on how many models misclassify it during cross-validation.
    """
    wrong_prediction_count = np.zeros(len(y))

    for model in models:
        y_pred = cross_val_predict(model, X, y, cv=cv)
        wrong_prediction_count += (y_pred != y).astype(int)

    # Define bins for difficulty categories (e.g. 0 wrong -> easy, 1-2 -> medium, etc.)
    bins = np.linspace(0, len(models), num=difficulty_categories+1)[1:-1]  # exclude first and last bin edges
    difficulty_labels = np.digitize(wrong_prediction_count, bins, right=False)

    # Map difficulty labels to sample indices
    difficulty_indices = {
        int(i): np.where(difficulty_labels == i)[0]
        for i in np.unique(difficulty_labels)
    }

    return wrong_prediction_count, difficulty_indices

In [ ]:
rfc = RandomForestClassifier(n_jobs = -1)
knc = KNeighborsClassifier(n_jobs = -1)
gbc = GradientBoostingClassifier()
svc = SVC()
mlp = MLP(hidden_layer_sizes=[16,16,16], alpha=0)
miss_count, indices = difficulty_split(X_train_resampled, y_train_resampled, models=[rfc, knc, gbc, svc, mlp], difficulty_categories=4)

In [ ]:
from sklearn.manifold import Isomap
import seaborn as sns
from matplotlib import pyplot as plt
def plot_difficulties_here(X, indices_map):
    iso = Isomap(n_components=2, n_jobs=-1)
    X_lowered = iso.fit_transform(X)
    
    color_map = {
        1: "green",
        2: "orange",
        3: "red"
    }
    
    plt.figure(figsize=(5, 5))
    
    for difficulty, idxs in indices_map.items():
        color = color_map.get(difficulty, "blue")  # Default color if difficulty is missing
        sns.scatterplot(x=X_lowered[idxs, 0], y=X_lowered[idxs, 1],
                        color=color, alpha=0.5, edgecolor='k', label=f'Difficulty {difficulty}')
    
    plt.legend()
    plt.xlabel("Isomap Dimension 1")
    plt.ylabel("Isomap Dimension 2")
    plt.title("Instance Difficulty Visualization")
    plt.show()

In [ ]:
plot_difficulties_here(X_train_resampled, indices)
for k in indices.keys():
    print(f"{len(indices[k])} instances in difficulty {k}.")


- [x] CV za train+test
- [x] Baseline performanse PO DIFFICULTYxMODEL
- [ ] Kako se sve dodaje shum?
- [ ] Smisli plot type kad se doda i shum
- [ ] Metode za uklanjanje suma

# Baseline performance

In [ ]:
def generate_test_pairs(X: pd.DataFrame, y: pd.DataFrame, models: list[ClassifierMixin], difficulty_indices:dict[int, int]) -> list[tuple[pd.DataFrame, pd.DataFrame, ClassifierMixin, str]]:
    results = list[tuple[pd.DataFrame, pd.DataFrame, ClassifierMixin, str]]()
    for model in models:
        for difficulty in difficulty_indices.keys():
            results.append(
                (X.iloc[difficulty_indices[difficulty]], y.iloc[difficulty_indices[difficulty]], model, f"{model.__class__.__name__} \n baseline performance \n difficulty level {difficulty}")
            )
    return results


In [ ]:
TestClassifiers(
    generate_test_pairs(X_train_resampled, y_train_resampled, models=[rfc, knc, gbc, svc, mlp], difficulty_indices=indices),
    n_columns = len(indices.keys())
)

# Dodavanje suma

- Generalizacija: da x% skupa podataka dodamo do y% suma od njegove vrednosti - 100% => random
    - Pri tom mozemo da primenimo samo nad atributima, samo nad klasama ili oba => biramo labele koje ce da dobiju noise 
        - Direktno navodjenjem, N labela kao maksimum ili metoda (sqrt, log nad brojem atributa)
        - Da mozemo da biramo kojim ce tezinama da se doda - da vidimo kako utice na pojedine klase
    - Pored toga mozemo da ili zamenimo vrednosti ili da ubacimo nove pa da vidimo kako se ponasaju -  da li ce da se prepoznaju i izbace lose instance

Znaci:
- Funkcija koja uzima:
    - X, Y skupovi, difficulty dict
    - x% - koliki deo skupa dobija sum
    - y% - koliki je procenat suma koji se dodaje ili dict `atribut:procenat suma za taj atribut`
    - apply_to - "all", "attributes", "classes"
    - labels - tacno koji atributi/klasa dobija sum ili N - broj nasumicnih labela koje dobijaju sum ili metoda
    - pre_pick_labels - da li prvo izabere random labels pa doda sum ili svaki put random se dodeljuje
    - replace - da li menjamo vrednosti ili nove ubacimo
    - difficulty_noise_schedule - `diff: procenat suma` ako drugacije sum primenjujemo nad razlicitim tezinama
- Funkcija vraca:
    - Staru kopiju podataka
    - Novu kopiju podataka
    - Indeksi instanci koje su menjane ukoliko stara=nova kopija ILI indeksi novo dodanih instanci



In [ ]:
def generate_noise(X: pd.DataFrame, 
                   Y: pd.DataFrame, 
                   difficulty_indices: dict[int, int],
                   instance_percentage: float,
                   noise_ratio: float,
                   attributes: list[str] | int | Callable[[int], int],
                   difficulty_noise_schedule: dict[int, float] | None,
                   replace: bool = True,
                   regenerate_labels: bool = False,
                   apply_to: Literal["both","attributes","classes"] = "attributes",
) -> tuple[pd.DataFrame, pd.DataFrame, list[int]]:
    """
        Generates a noisy version of the input dataset
        Inputs:
            X - the input attributes
            Y - the input classes
            difficulty_indices - dictionary containing indices for each of the difficulty categories
            instance_percentage - percentage of instances in the input (X and Y) that we add noise to
            noise_ratio - how much additional noise is added. 0% - no noise added, 100% - completely random value
            attributes - which attributes receive noise - list of names, number of attributes or a method (sqrt, log...)
            difficulty_noise_schedule - we might want to apply noise to specific percentage of instances with difficulty 0 and another to ones with difficulty 1
            replace - do we replace the values or add new instances?
            regenerate_labels - do we randomly pick labels every time we apply noise?
            apply_to - which columns get noise - attributes, classes or both
        Output:
            New values of X and Y with added noise
            List of indices of newly added or updated instances
    """

    